# 0.框架

In [ ]:
result = []
def backtrack(路径, 选择列表):
    if 满足结束条件:
        result.add(路径)
        return
    
    for 选择 in 选择列表:
        做选择
        backtrack(路径, 选择列表)
        撤销选择

## 1.数独游戏的回溯算法模板----二维坐标转一维索引---使用哈希表空间换时间(优化回溯算法的高时间复杂度)

In [ ]:
class Solution:
    def __init__(self):
        # 标记是否已经找到可行解
        self.found = False
        # 记录每行已经出现的数字
        # 比如 rows[0] = {1, 2, 3} 表示第 0 行已经出现了数字 1, 2, 3
        self.rows = [set() for _ in range(9)]
        # 记录每列已经出现的数字
        self.cols = [set() for _ in range(9)]
        # 记录每个九宫格已经出现的数字
        self.boxes = [set() for _ in range(9)]


    def solveSudoku(self, board):
        # 将预设数字加入集合
        for i in range(9):
            for j in range(9):
                if board[i][j] != '.':
                    self.rows[i].add(board[i][j])
                    self.cols[j].add(board[i][j])
                    self.boxes[self.getBoxIndex(i, j)].add(board[i][j])

        self.backtrack(board, 0)


    # 路径：board 中小于 index 的位置所填的数字
    # 选择列表：数字 1~9
    # 结束条件：整个 board 都填满数字
    def backtrack(self, board, index):
        if self.found:
            # 已经找到一个可行解，立即结束
            return

        m, n = 9, 9
        i, j = index // n, index % n
        if index == m * n:
            # 找到一个可行解，触发 base case
            self.found = True
            return

        if board[i][j] != '.':
            # 如果有预设数字，不用我们穷举
            self.backtrack(board, index + 1)
            return

        for ch in '123456789':
            # 剪枝：如果遇到不合法的数字，就跳过
            if not self.isValid(board, i, j, ch):
                continue

            # 做选择，把 ch 填入 board[i][j]
            board[i][j] = ch
            self.rows[i].add(ch)
            self.cols[j].add(ch)
            self.boxes[self.getBoxIndex(i, j)].add(ch)

            self.backtrack(board, index + 1)
            if self.found:
                # 如果找到一个可行解，立即结束
                # 不要撤销选择，否则 board[i][j] 会被重置为 '.'
                return

            # 撤销选择，把 board[i][j] 重置为 '.'
            board[i][j] = '.'
            self.rows[i].remove(ch)
            self.cols[j].remove(ch)
            self.boxes[self.getBoxIndex(i, j)].remove(ch)


    # 获取 (r, c) 所在的九宫格索引
    def getBoxIndex(self, r, c):
        return (r // 3) * 3 + c // 3

    # 判断是否可以在 (r, c) 位置放置数字 num
    def isValid(self, board, r, c, num):
        # 现在只需要查询三次哈希表即可
        if num in self.rows[r]:
            return False
        if num in self.cols[c]:
            return False
        if num in self.boxes[self.getBoxIndex(r, c)]:
            return False
        return True

## 2.N皇后问题
用**回溯算法**解决N皇后问题的经典实现，核心思路是通过递归穷举所有可能的皇后放置方案，同时通过“剪枝”（合法性检查）排除无效方案，最终收集所有合法解。我们可以从“问题拆解”“回溯框架”“细节实现”三个层面逐步理解。


### 一、问题核心：N皇后的规则
N皇后问题要求在`N×N`的棋盘上放置`N`个皇后，满足：
1. 每行**恰好1个**皇后；
2. 每列**恰好1个**皇后；
3. 每条对角线（左上-右下、右上-左下）**最多1个**皇后。

本质是“在约束条件下的穷举问题”，适合用回溯算法解决——因为需要尝试所有可能的放置方式，同时及时排除违反规则的路径。


### 二、解法整体思路：按行递归，逐列尝试
回溯算法的核心是“路径+选择列表+结束条件”，对应到这个问题：
- **路径**：已经放置好皇后的行（即`board`中`0~row-1`行的皇后位置）；
- **选择列表**：当前行`row`中可以放置皇后的列（`0~n-1`列，需通过合法性检查筛选）；
- **结束条件**：已经处理完所有行（`row == n`），此时`board`就是一个合法解。

算法流程：
1. 从第0行开始，逐行处理；
2. 对当前行`row`，尝试每一列`col`：
   - 若`(row, col)`位置合法（不与已有皇后冲突），则放置皇后；
   - 递归处理下一行（`row+1`）；
   - 递归返回后，“撤销选择”（移除当前行的皇后），尝试下一列；
3. 当所有行都处理完（`row == n`），将当前棋盘加入结果列表。


### 三、关键细节拆解
#### 1. 棋盘初始化：`board = ["." * n for _ in range(n)]`
- `board`是一个长度为`n`的列表，每个元素是长度为`n`的字符串（初始全为`.`）；
- 例如`n=4`时，初始`board = [". . . .", ". . . .", ". . . .", ". . . ."]`（空格仅为示意），表示空棋盘。


#### 2. 回溯函数`backtrack(board, row)`
作用：处理第`row`行，尝试在该行放置皇后，并递归处理后续行。

##### （1）结束条件：`if row == len(board)`
- 当`row`等于棋盘行数`n`时，说明所有行都已放置好皇后（0~n-1行），当前`board`是一个合法解；
- `self.res.append(board[:])`：将当前棋盘的**副本**加入结果（注意用`board[:]`而不是`board`，因为列表是引用类型，直接存`board`会被后续修改影响）。


##### （2）遍历当前行的所有列：`for col in range(n)`
- 对第`row`行，依次尝试每一列`col`，判断是否可以放置皇后。


##### （3）合法性检查：`if not self.isValid(board, row, col): continue`
- 若`(row, col)`位置不合法（与已有皇后冲突），直接跳过该列；
- 合法性检查的细节见下文。


##### （4）做选择：`board[row] = board[row][:col] + 'Q' + board[row][col+1:]`
- 由于字符串是不可变类型，通过切片方式将第`row`行的第`col`列改为`'Q'`（表示放置皇后）；
- 例如`row=0`，`col=1`，`n=4`时，`board[0]`从`"...."`变为`".Q.."`。


##### （5）递归处理下一行：`self.backtrack(board, row + 1)`
- 放置好当前行的皇后后，递归处理下一行（`row+1`），继续放置皇后。


##### （6）撤销选择：`board[row] = board[row][:col] + '.' + board[row][col+1:]`
- 递归返回后（无论下一行是否找到合法解），需要将当前行的皇后移除（改回`.`）；
- 目的是尝试当前行的下一列（例如当前列`col=1`失败后，恢复为`.`再试`col=2`），这是回溯算法“回溯”的核心——不保留无效选择，为其他路径让路。


#### 3. 合法性检查函数`isValid(board, row, col)`
作用：判断在`(row, col)`位置放置皇后是否合法（不与已有皇后冲突）。

需要检查三个方向（仅需检查**上方**的行，因为下方的行还未放置皇后）：

##### （1）检查同一列（列冲突）：
```python
for i in range(row):
    if board[i][col] == 'Q':
        return False
```
- 遍历`0~row-1`行（已放置皇后的行），若同一列`col`有皇后，则冲突。
- 例：`row=2, col=1`时，检查第0行和第1行的第1列是否有皇后。


##### （2）检查右上对角线（斜率为-1的对角线）：
```python
for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
    if board[i][j] == 'Q':
        return False
```
- 右上对角线的特点：行减小1，列增加1（如`(row, col) → (row-1, col+1) → (row-2, col+2)`）；
- 遍历范围：行从`row-1`向上到0，列从`col+1`向右到`n-1`；
- 例：`row=2, col=1`时，检查`(1,2)、(0,3)`是否有皇后。


##### （3）检查左上对角线（斜率为1的对角线）：
```python
for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
    if board[i][j] == 'Q':
        return False
```
- 左上对角线的特点：行减小1，列减小1（如`(row, col) → (row-1, col-1) → (row-2, col-2)`）；
- 遍历范围：行从`row-1`向上到0，列从`col-1`向左到0；
- 例：`row=2, col=1`时，检查`(1,0)`是否有皇后（因为`col-1=0`，再向左列索引为负，停止）。


### 四、举例理解：n=4时的执行过程
以`n=4`（4皇后问题）为例，简化步骤：
1. 初始`board`全为`.`，调用`backtrack(board, 0)`（处理第0行）；
2. 第0行尝试`col=0`：
   - 检查合法性：0~-1行（无），合法；
   - 放置皇后：`board[0] = "Q..."`；
   - 递归`backtrack(board, 1)`（处理第1行）；
3. 第1行尝试`col=0`：同一列有皇后（第0行col=0），不合法；
   - 尝试`col=1`：左上对角线`(0,0)`有皇后（行差1，列差1），不合法；
   - 尝试`col=2`：合法（无冲突），放置皇后：`board[1] = "..Q."`；
   - 递归`backtrack(board, 2)`（处理第2行）；
4. 第2行尝试各列，均冲突（自行验证），回溯：第1行撤销`col=2`，改试`col=3`；
   - 第1行`col=3`合法，放置皇后：`board[1] = "...Q"`；
   - 递归`backtrack(board, 2)`；
5. 第2行尝试`col=0`：合法，放置皇后：`board[2] = "Q..."`；
   - 递归`backtrack(board, 3)`（处理第3行）；
6. 第3行尝试各列，最终`col=2`合法，放置后`row=4 == n=4`，将当前`board`加入结果；
7. 后续继续回溯，尝试其他可能的列，收集所有合法解。


### 五、总结
这个解法的核心是：
- 按行递归，每行只放一个皇后，减少搜索空间；
- 通过`isValid`函数及时排除冲突位置（剪枝）；
- 用“做选择-递归-撤销选择”的回溯框架，穷举所有合法解。

回溯算法的本质是“试错”：尝试一种选择，若走不通就退回上一步换一种选择，直到找到所有可行解。N皇后问题是回溯算法的经典应用，理解它有助于掌握“约束条件下的穷举”思路。

### 六.代码内容

In [ ]:
class Solution:
    def __init__(self):
        self.res = []

    # 输入棋盘边长 n，返回所有合法的放置
    def solveNQueens(self, n: int) -> List[List[str]]:
        # '.' 表示空，'Q' 表示皇后，初始化空棋盘。
        board = ["." * n for _ in range(n)]
        self.backtrack(board, 0)
        return self.res

    # 路径：board 中小于 row 的那些行都已经成功放置了皇后
    # 选择列表：第 row 行的所有列都是放置皇后的选择
    # 结束条件：row 超过 board 的最后一行
    def backtrack(self, board: List[str], row: int) -> None:
        # 触发结束条件
        if row == len(board):
            self.res.append(board[:])
            return
        
        n = len(board)
        for col in range(n):
            # 排除不合法选择
            if not self.isValid(board, row, col):
                continue
            # 做选择
            board[row] = board[row][:col] + 'Q' + board[row][col+1:]
            # 进入下一行决策
            self.backtrack(board, row + 1)
            # 撤销选择
            board[row] = board[row][:col] + '.' + board[row][col+1:]

    # 是否可以在 board[row][col] 放置皇后？
    def isValid(self, board: List[str], row: int, col: int) -> bool:
        n = len(board)
        # 检查列是否有皇后互相冲突
        for i in range(row):
            if board[i][col] == 'Q':
                return False
        # 检查右上方是否有皇后互相冲突
        for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
            if board[i][j] == 'Q':
                return False
        # 检查左上方是否有皇后互相冲突
        for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
            if board[i][j] == 'Q':
                return False
        return True

## 3.排列/组合/子集问题

### 3.1元素无重不可复选

In [ ]:
from typing import List
# 组合/子集问题回溯算法框架
def backtrack(nums: List[int], start: int):
    # 回溯算法标准框架
    for i in range(start, len(nums)):
        # 做选择
        track.append(nums[i])
        # 注意参数
        backtrack(nums, i + 1)
        # 撤销选择
        track.pop()

# 排列问题回溯算法框架
def backtrack(nums: List[int]):
    for i in range(len(nums)):
        # 剪枝逻辑
        if used[i]:
            continue
        # 做选择
        used[i] = True
        track.append(nums[i])

        backtrack(nums)
        # 撤销选择
        track.pop()
        used[i] = False